In [ ]:
import tensorflow as tf
from tensorflow import keras

IMG_SIZE = 100

class Model(tf.Module):

    def __init__(self):
        # Load the base model with the correct input shape and remove the top layer
        base_model = keras.applications.DenseNet201(
            include_top=False,  # Remove the fully connected layers on top
            weights="imagenet",
            input_tensor=None,
            input_shape=(224, 224, 3),
            pooling=None,
        )
        base_model.trainable = False  # Freeze the base model

        # Build the custom model
        self.model = tf.keras.Sequential([
            # Input layer
            tf.keras.Input(shape=(100, 100, 3)),

            # Augmentation layers
            RandomCrop(height=100, width=100),  # RandomCrop to the input size (optional, can be adjusted)
            tf.keras.layers.Resizing(224, 224),  # Resize images to match ResNet101V2 input size
            RandomFlip(mode='horizontal'),
            RandomTranslation(height_factor=0.1, width_factor=0.1),
            RandomRotation(factor=0.2),
            RandomZoom(height_factor=0.1, width_factor=0.1),
            RandomContrast(factor=0.2),
            RandomBrightness(factor=0.2),

            # Base model
            base_model,

            # Global average pooling to reduce dimensions
            tf.keras.layers.GlobalAveragePooling2D(),

            # Fully connected layers
            tf.keras.layers.Dense(256, activation='relu'),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(64, activation='relu'),

            # Output layer for 3 classes
            tf.keras.layers.Dense(3, activation='softmax')
        ])

        self.model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=[tf.keras.metrics.CategoricalAccuracy()]
        )

    # The `train` function takes a batch of input images and labels.
    @tf.function(input_signature=[
        tf.TensorSpec([None, IMG_SIZE, IMG_SIZE, 3], tf.float32),
        tf.TensorSpec([None, 3], tf.float32)
    ])
    def train(self, x, y):
        with tf.GradientTape() as tape:
            prediction = self.model(x)
            loss = self.model.loss(y, prediction)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.model.optimizer.apply_gradients(
            zip(gradients, self.model.trainable_variables))
        result = {"loss": loss}
        return result

    @tf.function(input_signature=[
        tf.TensorSpec([None, IMG_SIZE, IMG_SIZE, 3], tf.float32),
    ])
    def infer(self, x):
        logits = self.model(x)
        probabilities = tf.nn.softmax(logits, axis=-1)
        return {
            "output": probabilities,
            "logits": logits
        }

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def save(self, checkpoint_path):
        tensor_names = [weight.name for weight in self.model.weights]
        tensors_to_save = [weight.read_value() for weight in self.model.weights]
        tf.raw_ops.Save(
            filename=checkpoint_path, tensor_names=tensor_names,
            data=tensors_to_save, name='save')
        return {
            "checkpoint_path": checkpoint_path
        }

    @tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
    def restore(self, checkpoint_path):
        restored_tensors = {}
        for var in self.model.weights:
            restored = tf.raw_ops.Restore(
                file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype,
                name='restore')
            var.assign(restored)
            restored_tensors[var.name] = restored
        return restored_tensors


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)
test2_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_dataset = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ chess/ dataset/train',  # Replace with the path to your training dataset
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'  # Ensure labels are one-hot encoded
)

test_dataset = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ chess/ dataset/test',  # Replace with the path to your validation dataset
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'  # Ensure labels are one-hot encoded
)

test2_dataset = test2_datagen.flow_from_directory(
    '/content/drive/MyDrive/ chess/ dataset/test2',  # Replace with the path to your training dataset
    target_size=(100, 100),
    batch_size=32,
    class_mode='categorical'  # Ensure labels are one-hot encoded
)

Found 1313 images belonging to 3 classes.
Found 110 images belonging to 3 classes.
Found 64 images belonging to 3 classes.


In [ ]:
# Extract a batch of data for tuning
train_img, train_labels = next(train_dataset)
test_img, test_labels = next(test_dataset)
test2_img, test2_labels = next(test2_dataset)


In [ ]:
model_instance = Model()

# Train the model
model_instance.model.fit(train_img,train_labels, epochs=1000, validation_split=0.2)

Epoch 1/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - categorical_accuracy: 0.3200 - loss: 1.1092 - val_categorical_accuracy: 0.7143 - val_loss: 0.8359
Epoch 2/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - categorical_accuracy: 0.5600 - loss: 0.8990 - val_categorical_accuracy: 0.4286 - val_loss: 0.8572
Epoch 3/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - categorical_accuracy: 0.3200 - loss: 0.9528 - val_categorical_accuracy: 0.7143 - val_loss: 0.7247
Epoch 4/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - categorical_accuracy: 0.5600 - loss: 1.0152 - val_categorical_accuracy: 0.2857 - val_loss: 0.9344
Epoch 5/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - categorical_accuracy: 0.3600 - loss: 1.0640 - val_categorical_accuracy: 0.7143 - val_loss: 0.7440
Epoch 6/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - categorical_accuracy: 0.5600 - loss: 0.9204 - val_categorical_accuracy: 0.7143 - val_loss: 0.7336
Epoch 7/1000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - categorical_accuracy: 0.5600 - loss

In [ ]:
model_instance.model.fit(train_img,train_labels, epochs=1000, validation_split=0.2)

Streaming output truncated to the last 5000 lines.
Epoch 501/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - categorical_accuracy: 0.6000 - loss: 0.8145 - val_categorical_accuracy: 1.0000 - val_loss: 0.0035
Epoch 502/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - categorical_accuracy: 0.6400 - loss: 0.7764 - val_categorical_accuracy: 1.0000 - val_loss: 0.0026
Epoch 503/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step - categorical_accuracy: 0.7200 - loss: 0.6671 - val_categorical_accuracy: 1.0000 - val_loss: 0.0023
Epoch 504/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - categorical_accuracy: 0.6000 - loss: 0.7378 - val_categorical_accuracy: 1.0000 - val_loss: 0.0020
Epoch 505/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - categorical_accuracy: 0.6400 - loss: 0.6790 - val_categorical_accuracy: 1.0000 - val_loss: 0.0014
Epoch 506/3000
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - categorical_accuracy: 0.6000 - loss: 0.8090 - val_categorical_accuracy: 1.0000 - val_loss: 0.0178
Epoch 507/3000
1/1 ━━━━━━